In [0]:
%sql
CREATE OR REPLACE STREAMING TABLE customers AS 
SELECT  
 *, 
 CURRENT_TIMESTAMP AS effective_start, 
 NULL AS effective_end, 
 'Y' AS is_current 
FROM STREAM READ_FILES( 
 "/Volumes/capstone_project/bronze/raw/customers",  
 FORMAT => "csv" 
); 

Silver pl :-
create or replace streaming table capstone_project.silver.customers_cleaned_pl
as
select distinct * from stream customers where customer_id is not null;

CREATE MATERIALIZED VIEW masked_silver AS
SELECT  
*, 
concat('xxxx', '@', split(email, '@')[1]) AS masked_email 
FROM  
capstone_project.silver.customers_cleaned_pl; 

CREATE OR REFRESH STREAMING TABLE capstone_project.silver.customers_cleaned_pl; 

CREATE FLOW customers_flow 
AS AUTO CDC INTO 
 capstone_project.silver.customers_cleaned_pl 
FROM stream(masked_silver) 
 KEYS (customer_id) 
 APPLY AS DELETE WHEN operation = "DELETE" 
 SEQUENCE BY customer_id 
 COLUMNS * EXCEPT (operation, sequenceNum,_rescued_data,ingestion_date) 
 STORED AS SCD TYPE 2; 

MERGE INTO capstone_project.silver.customers_cleaned_pl AS target 
USING stream(masked_silver) AS source 
ON target.customer_id = source.customer_id 
WHEN MATCHED AND source.operation = "UPDATE" THEN 
 UPDATE SET  
   target.effective_end = CURRENT_TIMESTAMP, 
   target.is_current = 'N' 
WHEN NOT MATCHED THEN 
 INSERT ( 
   customer_id, 
   name, 
   email, 
   phone, 
   address, 
   effective_start, 
   effective_end, 
   is_current 
 ) VALUES ( 
   source.customer_id, 
   source.name, 
   source.email, 
   source.phone, 
   source.address, 
   CURRENT_TIMESTAMP, 
   NULL, 
   'Y' 
 ); 


In [0]:
%sql
select * from capstone_project.silver.customers_clean_pl ;



In [0]:
%sql
select * from capstone_project.silver.masked_customers_email1 limit(10);

In [0]:
%sql
describe capstone_project.bronze.customers1

In [0]:
%sql
select * from capstone_project.bronze.customers 
    


In [0]:
%sql
select * from capstone_project.silver.customers_scd2_pl1 limit(10);

In [0]:
%sql
describe capstone_project.silver.customers_SCD2_pl1

In [0]:
%sql
select * from capstone_project.silver.customers_SCD2 limit(10)

In [0]:
%sql
select * from capstone_project.silver.masked_email_customers

In [0]:
%sql
select * from capstone_project.bronze.accounts limit(10)

In [0]:
%sql
describe capstone_project.bronze.loans

In [0]:
%sql
describe capstone_project.bronze.accounts 

In [0]:
%sql
select * from capstone_project.bronze.loans limit(10)

In [0]:
%sql
select * from capstone_project.bronze.transactions limit(10)

In [0]:
%sql
describe capstone_project.bronze.transactions

In [0]:
%sql
select * from capstone_project.bronze.branches limit(10)
    


In [0]:
Masking on same “email” column

CREATE OR REPLACE STREAMING TABLE silver.masked_customers_email 
AS 
SELECT  
 customer_id, 
 first_name, 
 last_name, 
 concat('xxxx', '@', split(email, '@')[1]) AS email, 
 phone, 
 date_of_birth, 
 gender, 
 annual_income, 
 pan_number, 
 aadhar_number, 
 city, 
 state, 
 pincode, 
 customer_since, 
 kyc_status, 
 credit_score, 
 risk_category, 
 is_active 
FROM  
 STREAM(silver.customers_cleaned_pl); 
